In [1]:
import os
import sys
import pickle

import cv2
import numpy as np
import neuroglancer
from tqdm import tqdm
from cloudvolume import CloudVolume

sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.SqlController import SqlController
from utils import get_structure_number, get_segment_properties, NumpyToNeuroglancer

Connecting dklab@192.168.1.12:3306


In [2]:
animal = 'MD589'
downsample_factor = 16

OUTPUT_DIR_PATH = os.path.join('./')
IMAGE_DIR_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/full'
NG_DIR_PATH = os.path.join('/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures')
# NG_DIR_PATH = os.path.join('./')

sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
aligned_shape = np.array((sqlController.scan_run.width, sqlController.scan_run.height))
num_section = len(os.listdir(IMAGE_DIR_PATH))

downsampled_aligned_shape = np.round(aligned_shape / downsample_factor).astype(int)
scales = np.array([resolution * downsample_factor, resolution * downsample_factor, 20]) * 1000

In [3]:
def draw_numpy(section_structure_polygons, section_start, section_end, draw_lines=True):
    volume = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0], section_end - section_start), dtype=np.uint8)
    for section in tqdm(range(section_start, section_end)):
        if section in section_structure_polygons:
            template = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0]), dtype=np.uint8)
            for structure in section_structure_polygons[section]:
                polygons = section_structure_polygons[section][structure]
                for polygon in polygons:
                    color = get_structure_number(structure)
                    
                    if draw_lines:
                        cv2.polylines(template, [polygon.astype(np.int32)], True, color, 1)
                    else:
                        for point in polygon:
                            cv2.circle(template, tuple(point.astype(np.int32)), 0, color, -1)

            volume[:, :, section - section_start - 1] = template
        
    volume = np.swapaxes(volume, 0, 1)
    return volume

def get_ranges(start, end, interval):
    ranges = []
    
    cur = start
    while cur + interval <= end:
        next = cur + interval
        ranges.append((cur, next))
        cur = next
    ranges.append((cur, end))
    return ranges

In [4]:
draw_lines = True
polygons_path = os.path.join(OUTPUT_DIR_PATH, f'MD589_CVAT_downsample{downsample_factor}.pkl')

with open(polygons_path, 'rb') as file:
    section_structure_polygons = pickle.load(file)
volume = draw_numpy(section_structure_polygons, 0, num_section, draw_lines=draw_lines)

FileNotFoundError: [Errno 2] No such file or directory: './MD589_CVAT_downsample16.pkl'

In [5]:
precompute_path = os.path.join(NG_DIR_PATH, f'{animal}_down{downsample_factor}_annotations')

ng = NumpyToNeuroglancer(volume, scales)
ng.preview()
ng.init_precomputed(precompute_path)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()

NameError: name 'volume' is not defined